In [ ]:
import pandas as pd

def determine_thresholds(df, usd_col='USD_amount'):
    # Calculate percentiles (25th, 50th, 75th, 90th, 100th)
    percentiles = df[usd_col].quantile([0.25, 0.5, 0.75, 0.9, 1.0]).values
    thresholds = {
        'Analyst': percentiles[0],  # 25th percentile
        'Manager': percentiles[1],   # 50th percentile
        'AVP': percentiles[2],       # 75th percentile
        'VP': percentiles[3],        # 90th percentile
        'Senior Management': percentiles[4]  # 100th percentile
    }
    
    # Count how many trades fall under each threshold
    summary = {}
    for role, threshold in thresholds.items():
        count = df[df[usd_col] < threshold].shape[0]
        summary[role] = {
            'Threshold': threshold,
            'Trade_Count': count
        }
    
    return thresholds, summary

def filter_and_determine_thresholds(df, instrument_group, usd_col='USD_amount'):
    # Filter by Instrument Group
    filtered_df = df[df['Instrument_Group'] == instrument_group]
    
    if len(filtered_df) > 0:
        # Determine thresholds and summary for the filtered data
        return determine_thresholds(filtered_df, usd_col)
    else:
        return None, None

def process_and_save_to_excel(df, output_filename):
    with pd.ExcelWriter(output_filename) as writer:
        # Define the instrument groups
        instrument_groups = ['Equity', "'WARR', 'debt'", 'CURR']
        
        # Loop over each Instrument Group and calculate thresholds
        for group in instrument_groups:
            thresholds, summary = filter_and_determine_thresholds(df, group)
            if thresholds is not None:
                # Create a summary DataFrame
                summary_df = pd.DataFrame.from_dict(summary, orient='index')
                summary_df.index.name = 'Role'
                
                # Save each group result in a separate sheet
                summary_df.to_excel(writer, sheet_name=group, index=True)

# Example usage
df = pd.DataFrame({
    'Instrument_Group': ['Equity', 'Equity', 'WARR, debt', 'CURR', 'Equity', 'CURR', 'WARR, debt'],
    'USD_amount': [100000, 600000, 1200000, 7000000, 300000, 200000, 900000],
})

# Process the data and save results to Excel
process_and_save_to_excel(df, 'trade_thresholds.xlsx')


In [ ]:
import pandas as pd

def determine_quantiles_and_top_counterparties(df, usd_col='USD_amount', counterparty_col='Counterparty'):
    # Calculate percentiles (25th, 50th, 75th, 90th, 100th)
    quantiles = df[usd_col].quantile([0.25, 0.5, 0.75, 0.9, 1.0]).values
    
    # Create a DataFrame for quantiles
    quantile_df = pd.DataFrame(quantiles, columns=['Quantile'])
    
    # Add top 5 counterparties for each quantile
    top_counterparties = []
    for q in quantiles:
        top_5 = df[df[usd_col] <= q][counterparty_col].value_counts().head(5).index.tolist()
        top_counterparties.append(', '.join(top_5))
    
    quantile_df['Top_Counterparties'] = top_counterparties
    
    return quantile_df

def filter_and_process_quantiles(df, instrument_group, usd_col='USD_amount', counterparty_col='Counterparty'):
    # Filter by Instrument Group using isin
    filtered_df = df[df['Instrument_Group'].isin(instrument_group)]
    
    if len(filtered_df) > 0:
        # Determine quantiles and top counterparties for the filtered data
        return determine_quantiles_and_top_counterparties(filtered_df, usd_col, counterparty_col)
    else:
        return None

def process_and_save_to_excel(df, output_filename):
    with pd.ExcelWriter(output_filename) as writer:
        # Define the instrument groups
        instrument_groups = {
            'Equity': ['Equity'],
            'WARR, debt': ['WARR, debt'],  # Group as is for filtering
            'CURR': ['CURR']
        }
        
        # Loop over each Instrument Group and calculate quantiles and top counterparties
        for group_name, group_values in instrument_groups.items():
            quantile_df = filter_and_process_quantiles(df, group_values)
            if quantile_df is not None:
                # Save each group result in a separate sheet
                quantile_df.to_excel(writer, sheet_name=group_name, index=False)

# Example usage
df = pd.DataFrame({
    'Instrument_Group': ['Equity', 'Equity', 'WARR, debt', 'CURR', 'Equity', 'CURR', 'WARR, debt'],
    'USD_amount': [100000, 600000, 1200000, 7000000, 300000, 200000, 900000],
    'Counterparty': ['A', 'B', 'A', 'C', 'B', 'D', 'A'],
})

# Process the data and save results to Excel
process_and_save_to_excel(df, 'trade_quantiles.xlsx')
